# Проверка работоспособности.

Я написал 10 моделей one vs all для каждого класса.

Сейчас мы попробуем из всех них составить единый файл с ответам и закинуть на kaggle.

импорт необходимых библиотек

In [1]:
import sys

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

sys.path.append('../../../')

from core.datasets import open_f

c:\Users\Mi\Desktop\ML\.conda\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
class MulticlassClassificationMetrics:
    def __init__(self, y_true, y_pred):
        self.y_true = np.array(y_true)
        self.y_pred = np.array(y_pred)
        self.matrix_error = self.get_matrix_error()

    def get_matrix_error(self):
        self.class_types = np.unique([self.y_true, self.y_pred])
        TP, FN, FP, TN = [], [], [], []
        for class_type in self.class_types:
            TP.append(np.sum(np.logical_and(self.y_true == self.y_pred, self.y_true == class_type)))
            FN.append(np.sum(np.logical_and(self.y_true == class_type, self.y_pred != class_type)))
            FP.append(np.sum(np.logical_and(self.y_pred == class_type, self.y_true != class_type)))
            TN.append(np.sum(np.logical_and(self.y_pred != class_type, self.y_true != class_type)))
        return pd.DataFrame({
            'class_type': self.class_types,
            'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN,
        }).set_index('class_type')

    def accuracy(self):
        return np.sum(self.y_true == self.y_pred) / self.y_true.shape[0]

    def precision(self, averaging='macro'):
        if averaging == 'macro':
            precisions = self.matrix_error['TP'] / (self.matrix_error['TP'] + self.matrix_error['FP'])
            return np.mean(precisions.replace(np.nan, 0))
        elif averaging == 'micro':
            mean_val = self.matrix_error.mean()
            return mean_val['TP'] / (mean_val['TP'] + mean_val['FP'])

    def recall(self, averaging='macro'):
        if averaging == 'macro':
            recalls = self.matrix_error['TP'] / (self.matrix_error['TP'] + self.matrix_error['FN'])
            return np.mean(recalls.replace(np.nan, 0))
        elif averaging == 'micro':
            mean_val = self.matrix_error.mean()
            return mean_val['TP'] / (mean_val['TP'] + mean_val['FN'])

    def f1_score(self, averaging='macro'):
        if averaging == 'macro':
            reverse_r = (self.matrix_error['TP'] + self.matrix_error['FN']) / self.matrix_error['TP']
            reverse_p = (self.matrix_error['TP'] + self.matrix_error['FP']) / self.matrix_error['TP']
            f1_scores = 2 / (reverse_r + reverse_p)
            return np.mean(f1_scores.replace(np.nan, 0))
        elif averaging == 'micro':
            p = self.precision(averaging)
            r = self.recall(averaging)
            return 2 * p * r / (p + r)

    def metrics(self, averaging='macro'):
        return {
            'accuracy': self.accuracy(),
            'precision': self.precision(averaging),
            'recall': self.recall(averaging),
            'f1_score': self.f1_score(averaging),
        }

    def __str__(self):
        headers = {
            'selector': '*',
            'props': 'background-color: darkgreen; color: white; font-size: 12pt;',
        }
        che_super_mega_puper_visualization_of_cell = { 
            'selector': 'td:hover',
            'props': 'background-color: green; color: white;',
        }
        display(
            self.matrix_error
            .style
            .set_table_styles([
                headers,
                che_super_mega_puper_visualization_of_cell,
            ])
            .set_properties(**{'background-color': 'lightgreen',
                           'color': 'black', 'font-size': '12pt'})
        )
        return ''

    def __repr__(self):
        return (
            f'MulticlassClassificationMetrics(class_types={self.class_types})'
            .replace("'", '').replace(',', ';'))

Скачиваем тестовый датасет

In [3]:
test_ds = open_f('repaired_data_train', back=3)
shuffle = np.random.permutation(test_ds['labels'].shape[0])
test_ds_x = test_ds['images'][shuffle][:10000] / 255
test_ds_y = test_ds['labels'][shuffle][:10000]

# Рассмотрим все avg acc чекпоинты 

In [4]:
nums = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
models = []
for i in nums:
    models.append(tf.keras.models.load_model(f'../../checkpoints/model_{i}_avg_categorical_accuracy.h5'))

In [7]:
ans = None
for i in models:
    pred = i.predict(test_ds_x).ravel()
    if type(ans) is type(None):
        ans = pred
    else:
        ans = np.column_stack((ans, pred))

101/313 [========>.....................] - ETA: 49s

In [ ]:
full_ans = np.argmax(ans, axis=-1)

In [ ]:
metrics = MulticlassClassificationMetrics(test_ds_y, full_ans)
print(metrics)
metrics.metrics()

# Сперва рассмотрим все acc чекпоинты 

In [ ]:
nums = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
models = []
for i in nums:
    models.append(tf.keras.models.load_model(f'../../checkpoints/model_{i}_categorical_accuracy.h5'))

In [ ]:
ans = None
for i in models:
    pred = i.predict(test_ds_x).ravel()
    if type(ans) is type(None):
        ans = pred
    else:
        ans = np.column_stack((ans, pred))

In [ ]:
full_ans = np.argmax(ans, axis=-1)

In [ ]:
metrics = MulticlassClassificationMetrics(test_ds_y, full_ans)
print(metrics)
metrics.metrics()

# Рассмотрим все avg loss чекпоинты 

In [ ]:
nums = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
models = []
for i in nums:
    models.append(tf.keras.models.load_model(f'../../checkpoints/model_{i}_avg_loss.h5'))

In [ ]:
ans = None
for i in models:
    pred = i.predict(test_ds_x).ravel()
    if type(ans) is type(None):
        ans = pred
    else:
        ans = np.column_stack((ans, pred))

In [ ]:
full_ans = np.argmax(ans, axis=-1)

In [ ]:
metrics = MulticlassClassificationMetrics(test_ds_y, full_ans)
print(metrics)
metrics.metrics()

# Рассмотрим все loss чекпоинты

In [ ]:
nums = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
models = []
for i in nums:
    models.append(tf.keras.models.load_model(f'../../checkpoints/model_{i}_loss.h5'))

In [ ]:
ans = None
for i in models:
    pred = i.predict(test_ds_x).ravel()
    if type(ans) is type(None):
        ans = pred
    else:
        ans = np.column_stack((ans, pred))

In [ ]:
full_ans = np.argmax(ans, axis=-1)

In [ ]:
metrics = MulticlassClassificationMetrics(test_ds_y, full_ans)
print(metrics)
metrics.metrics()